In [ ]:
from dichasus_cf0x import training_set
import tensorflow as tf
import numpy as np
import tqdm

In [ ]:
# Count number of datapoints in training set (for progress bar)
TOTAL_DATAPOINTS = sum(1 for _ in training_set)

In [ ]:
SUBCARRIERS = tf.shape(training_set.take(1).get_single_element()[0])[-1].numpy()

In [ ]:
# Rissanen MDL, as described in
# Xinrong Li and Kaveh Pahlavan: "Super-resolution TOA estimation with diversity for indoor geolocation" in IEEE Transactions on Wireless Communications
def rissanen_mdl(eigenvalues, chunkcount, use_fbcm = False, L = 100):
    eigenvalues = np.sort(np.real(eigenvalues))[::-1]
    
    M = chunkcount
    mdl = np.zeros(L)

    for k in range(L):
        mdl[k] = -M * (L - k) * (np.sum(np.log(eigenvalues[k:L]) / (L - k)) - np.log(np.sum(eigenvalues[k:L]) / (L - k)))
        if use_fbcm:
            mdl[k] = mdl[k] + (1/4) * k * (2 * L - k + 1) * np.log(M)
        else:
            mdl[k] = mdl[k] + (1/2) * k * (2 * L - k) * np.log(M)

    return np.argmin(mdl)

# root-MUSIC algorithm implementation
# returns delays and powers, sorted by power from strongest to weakest
def rootmusic_toa(eigval, eigvec, source_count):
    Qn = np.asmatrix(eigvec[:,source_count:])
    C = np.matmul(Qn, Qn.H)

    coeffs = np.zeros((len(eigval) - 1,), dtype = np.complex128)
    for i in range(1, len(eigval)):
        coeffs[i - 1] += np.sum(np.diag(C, i))

    coeffs = np.hstack((coeffs[::-1], np.sum(np.diag(C)), coeffs.conj()))
    roots = np.roots(coeffs)
    roots = roots[abs(roots) < 1]
    powers = 1 / (1 - np.abs(roots))
    largest_roots = np.argsort(powers)[::-1]

    source_delays = -SUBCARRIERS * np.angle(roots[largest_roots[:source_count]]) / (2 * np.pi)
    source_powers = powers[largest_roots[:source_count]]

    return source_delays, source_powers


In [ ]:
# Expects csi in shape (arrays, antenna_rows, antenna_columns, subcarriers)
def estimate_toas(csi, chunksize = None):
    chunksize = np.shape(csi)[-1] if chunksize is None else chunksize
    chunkcount = np.shape(csi)[-1] // chunksize

    # Compute array covariance matrix R and perform eigenvector / eigenvalue decomposition
    csi_chunked = np.reshape(csi, (np.shape(csi)[0], np.shape(csi)[1], np.shape(csi)[2], chunkcount, chunksize))
    R = np.einsum("armcs,armct->ast", csi_chunked, np.conj(csi_chunked))

    # Use forward–backward correlation matrix
    R = (R + np.flip(np.conj(R), axis = (1, 2))) / 2
    eigval, eigvec = np.linalg.eigh(R)
    eigval = eigval[:,::-1]
    eigvec = eigvec[:,:,::-1]

    toa_by_array = np.zeros(np.shape(csi)[0])
    for array in range(np.shape(csi)[0]):
        source_count = rissanen_mdl(eigval[array,:], chunkcount, use_fbcm = True)
        delays, powers = rootmusic_toa(eigval[array], eigvec[array], source_count)
    
        # Out of the strongest "source_count // 2" paths (or at least 1, but maximum 5), pick the earliest one
        if len(delays) > 0:
            toa_by_array[array] = np.min(delays[:min(5, max(source_count // 2, 1))])

    return toa_by_array

In [ ]:
estimated_toas = []

for csi, pos, time in tqdm.tqdm(training_set.take(10), total = TOTAL_DATAPOINTS):
    estimated_toas.append(estimate_toas(csi, chunksize = 256))

In [ ]:
estimated_toas = np.asarray(estimated_toas)

In [ ]:
np.save("results/estimated_toas.npy", estimated_toas)